In [ ]:
!pip install -q transformers datasets evaluate accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.2 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
import evaluate

In [ ]:
dataset_en = load_dataset("cardiffnlp/tweet_topic_multilingual", "en_2022")
print(dataset_en)

# Chia nhỏ thành train/validation
split = dataset_en["train"].train_test_split(test_size=0.2, seed=42)
train_dataset = split["train"]
eval_dataset = split["test"]



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/60 [00:00<?, ?it/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.56M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/177k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10140 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1127 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label_name_flatten', 'label', 'label_name'],
        num_rows: 10140
    })
    validation: Dataset({
        features: ['id', 'text', 'label_name_flatten', 'label', 'label_name'],
        num_rows: 1127
    })
})


In [ ]:
print(train_dataset[0])

{'id': '1223934804376207365', 'text': 'I really hope @user and @Sidharth_Shukla will be together soon.. I request u both not to fight.. Ur fans want to see uh both together not fighting with each other.. Love uh both..❣️ #SidNaazForever  #Shehnaazians  #SidharthShukla', 'label_name_flatten': 'Celebrity & Pop Culture, Music', 'label': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 'label_name': ['Celebrity & Pop Culture', 'Music']}


In [ ]:
label_names = dataset_en["train"].features["label"].feature.names
label2id = {name: i for i, name in enumerate(label_names)}
id2label = {i: name for i, name in enumerate(label_names)}
num_labels = len(label_names)

In [ ]:
model_name = "cardiffnlp/tweet-topic-base-multilingual"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    problem_type="multi_label_classification",
    id2label=id2label,
    label2id=label2id
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
    labels_batch = []
    for example_labels_list in examples["label"]:
        label_tensor = torch.zeros(num_labels, dtype=torch.float32)
        for label_index in example_labels_list:
            if label_index < num_labels:
                label_tensor[label_index] = 1.0
        labels_batch.append(label_tensor)

    tokenized_inputs["labels"] = labels_batch # Still a list of tensors here

    return tokenized_inputs

# Modify the map function to ensure the labels are tensors in the dataset
def prepare_labels_tensor(examples):
    examples["labels"] = [torch.tensor(label, dtype=torch.float32) for label in examples["labels"]]
    return examples


train_dataset = train_dataset.map(tokenize_function, batched=True)
train_dataset = train_dataset.map(prepare_labels_tensor, batched=True)


eval_dataset = eval_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(prepare_labels_tensor, batched=True)

Map:   0%|          | 0/8112 [00:00<?, ? examples/s]

Map:   0%|          | 0/8112 [00:00<?, ? examples/s]

Map:   0%|          | 0/2028 [00:00<?, ? examples/s]

Map:   0%|          | 0/2028 [00:00<?, ? examples/s]

In [ ]:
label_tensor = torch.tensor(train_dataset[0]["labels"])
print(label_tensor.dtype)


torch.float32


In [ ]:
for i in range(10):
    label = train_dataset[i]["labels"]
    print(f"Sample {i}: type = {type(label)}, dtype = {torch.tensor(label).dtype}")


Sample 0: type = <class 'list'>, dtype = torch.float32
Sample 1: type = <class 'list'>, dtype = torch.float32
Sample 2: type = <class 'list'>, dtype = torch.float32
Sample 3: type = <class 'list'>, dtype = torch.float32
Sample 4: type = <class 'list'>, dtype = torch.float32
Sample 5: type = <class 'list'>, dtype = torch.float32
Sample 6: type = <class 'list'>, dtype = torch.float32
Sample 7: type = <class 'list'>, dtype = torch.float32
Sample 8: type = <class 'list'>, dtype = torch.float32
Sample 9: type = <class 'list'>, dtype = torch.float32


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
f1_metric = evaluate.load("f1")

In [ ]:
!pip install -q scikit-learn
from sklearn.metrics import f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))      # sigmoid
    preds = (probs >= 0.3).astype(int)
    labels = labels.astype(int)
    return {
        "f1_micro": f1_score(labels, preds, average="micro", zero_division=0),
        "f1_macro": f1_score(labels, preds, average="macro", zero_division=0),
    }


In [ ]:
training_args = TrainingArguments(
    output_dir="./reddit_topic_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_micro",
    greater_is_better=True,
    fp16=False,
    report_to="none"
)

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Union

@dataclass
class CustomDataCollatorWithPadding(DataCollatorWithPadding):
    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        batch = super().__call__(features)
        # Ensure labels are of type torch.float32
        if "labels" in batch:
            batch["labels"] = batch["labels"].to(torch.float32)
        return batch

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=CustomDataCollatorWithPadding(tokenizer=tokenizer), # Use custom data collator
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

trainer.save_model("./reddit_topic_model")
tokenizer.save_pretrained("./reddit_topic_model")

print("Training complete. Model saved at ./reddit_topic_model")

Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro
1,0.086600,0.078207,0.836805,0.781106
2,0.072400,0.076916,0.839146,0.785179
3,0.061500,0.078841,0.830729,0.773134
4,0.053500,0.076680,0.836837,0.781544
5,0.048100,0.077342,0.834683,0.780850


Training complete. Model saved at ./reddit_topic_model


In [ ]:
# Mapping từ label index -> tên chủ đề
id2label = {
    0: "Arts & Culture",
    1: "Business & Entrepreneurs",
    2: "Celebrity & Pop Culture",
    3: "Diaries & Daily Life",
    4: "Family",
    5: "Fashion & Style",
    6: "Film, TV & Video",
    7: "Fitness & Health",
    8: "Food & Dining",
    9: "Gaming",
    10: "Learning & Educational",
    11: "Music",
    12: "News & Social Concern",
    13: "Other Hobbies",
    14: "Relationships",
    15: "Science & Technology",
    16: "Sports",
    17: "Travel & Adventure",
    18: "Youth & Student Life"
}

# Load model đã train
model_path = "./reddit_topic_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

# Hàm predict topic
def predict_topic(text, threshold=0.3):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.sigmoid(outputs.logits).squeeze().cpu().numpy()

    # chọn topic có xác suất >= threshold
    predicted_indices = [i for i, p in enumerate(probs) if p >= threshold]
    predicted_topics = [id2label[i] for i in predicted_indices]

    return predicted_topics, probs

# ---- Test thử ----
sample_texts = [
    "I really enjoy watching the new Marvel movies and TV shows.",
    "Just finished my morning workout, feeling great!",
    "Bitcoin and crypto markets are going crazy this week.",
    "We are planning a family trip to Japan next summer.",
    "Taylor Swift released a new song and it's amazing!"
]

for text in sample_texts:
    topics, probs = predict_topic(text)
    print(f"\nText: {text}")
    print(f"Predicted topics: {topics}")



Text: I really enjoy watching the new Marvel movies and TV shows.
Predicted topics: ['Film, TV & Video']

Text: Just finished my morning workout, feeling great!
Predicted topics: ['Diaries & Daily Life', 'Fitness & Health']

Text: Bitcoin and crypto markets are going crazy this week.
Predicted topics: ['Business & Entrepreneurs', 'News & Social Concern']

Text: We are planning a family trip to Japan next summer.
Predicted topics: ['Family', 'Travel & Adventure']

Text: Taylor Swift released a new song and it's amazing!
Predicted topics: ['Celebrity & Pop Culture', 'Music']


In [ ]:
!zip -r sub topic_reddit.zip reddit_topic_model/

	zip warning: name not matched: topic_reddit.zip
  adding: reddit_topic_model/ (stored 0%)
  adding: reddit_topic_model/tokenizer.json (deflated 76%)
  adding: reddit_topic_model/training_args.bin (deflated 54%)
  adding: reddit_topic_model/sentencepiece.bpe.model (deflated 49%)
  adding: reddit_topic_model/model.safetensors (deflated 7%)
  adding: reddit_topic_model/config.json (deflated 58%)
  adding: reddit_topic_model/checkpoint-1014/ (stored 0%)
  adding: reddit_topic_model/checkpoint-1014/tokenizer.json (deflated 76%)
  adding: reddit_topic_model/checkpoint-1014/training_args.bin (deflated 54%)
  adding: reddit_topic_model/checkpoint-1014/sentencepiece.bpe.model (deflated 49%)
  adding: reddit_topic_model/checkpoint-1014/model.safetensors (deflated 7%)
  adding: reddit_topic_model/checkpoint-1014/config.json (deflated 58%)
  adding: reddit_topic_model/checkpoint-1014/rng_state.pth (deflated 26%)
  adding: reddit_topic_model/checkpoint-1014/trainer_state.json (deflated 62%)
  addi

In [ ]:
from google.colab import files
files.download('sub.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>